In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import tensorflow as tf


import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from nltk import ngrams

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score

from collections import Counter
from itertools import chain

In [73]:
data = pd.read_csv('/content/drive/MyDrive/NLP/lab4/train.csv')

In [74]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [76]:
data = data[['text','target']]
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [77]:
disasterTweets = data[data['target'] == 1]
normalTweets = data[data['target'] == 0]

In [78]:
disasterTweets.head(10)


,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
5,#RockyFire Update => California Hwy. 20 closed...,1
6,#flood #disaster Heavy rain causes flash flood...,1
7,I'm on top of the hill and I can see a fire in...,1
8,There's an emergency evacuation happening now ...,1
9,I'm afraid that the tornado is coming to our a...,1


In [79]:
normalTweets.head(10)

,text,target
15,What's up man?,0
16,I love fruits,0
17,Summer is lovely,0
18,My car is so fast,0
19,What a goooooooaaaaaal!!!!!!,0
20,this is ridiculous....,0
21,London is cool ;),0
22,Love skiing,0
23,What a wonderful day!,0
24,LOOOOOOL,0


In [80]:
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    tokens = word_tokenize(tweet)  # Tokenize the tweet into words
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize each word
    lemmatized_tweet = ' '.join(lemmatized_tokens)  # Join the lemmatized tokens back into a sentence
    return lemmatized_tweet


disasterTweets['lemmatized_text'] = disasterTweets['text'].apply(lemmatize_tweet)
normalTweets['lemmatized_text'] = normalTweets['text'].apply(lemmatize_tweet)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-80-f09e06d9e778>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disasterTweets['lemmatized_text'] = disasterTweets['text'].apply(lemmatize_tweet)
<ipython-input-80-f09e06d9e778>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalTweets['lemmatized_text'] = normal

In [81]:
print("\n\nLemmatized Disaster Tweets:")
disasterTweets.head()



Lemmatized Disaster Tweets:


,text,target,lemmatized_text
0,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this # earthquake ...
1,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask . Canada
2,All residents asked to 'shelter in place' are ...,1,All resident asked to 'shelter in place ' are ...
3,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive # wildfire evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby # Alaska a ...


In [82]:
print("\nLemmatized Normal Tweets:")
normalTweets.head()


Lemmatized Normal Tweets:


,text,target,lemmatized_text
15,What's up man?,0,What 's up man ?
16,I love fruits,0,I love fruit
17,Summer is lovely,0,Summer is lovely
18,My car is so fast,0,My car is so fast
19,What a goooooooaaaaaal!!!!!!,0,What a goooooooaaaaaal ! ! ! ! ! !


In [83]:
def word_occurrences(tweet_list):
    # Tokenize the lemmatized text into individual words
    words = [word for tweet in tweet_list for word in tweet.split()]
    # the occurrences count
    word_counts = Counter(words)
    return word_counts

# Count word occurrences in disaster tweets
disasterTweets_count = word_occurrences(disasterTweets['lemmatized_text'])

normalTweets_count = word_occurrences(normalTweets['lemmatized_text'])

print("Disaster Tweets:")
print(disasterTweets_count)
print("\nNormal Tweets:")
print(normalTweets_count)

Disaster Tweets:
Counter({':': 3676, 'http': 2513, '#': 1703, '.': 1086, 'the': 1060, 'in': 1046, '@': 905, 'a': 892, 'of': 890, '?': 762, 'to': 710, '...': 592, 'and': 469, '-': 398, 'on': 392, 'for': 371, '!': 330, 'I': 329, 'is': 315, "'": 303, 'at': 275, "'s": 272, 'The': 259, ';': 251, 'by': 245, 'it': 209, 'from': 208, 'fire': 200, 'that': 185, 'A': 172, 'wa': 169, '&': 164, 'with': 163, 'are': 151, ')': 144, '(': 142, 'after': 140, 'have': 137, 'amp': 135, 'you': 119, 'via': 119, "n't": 117, 'this': 114, 'over': 110, 'California': 109, 'my': 107, 'be': 104, 'ha': 102, 'people': 94, 'been': 94, 'up': 94, 'In': 92, 'who': 91, 'were': 90, 'News': 90, 'an': 89, 'killed': 87, 'like': 87, 'Hiroshima': 85, 'than': 82, 'building': 81, 'not': 80, 'do': 77, 'when': 77, '2': 76, 'family': 76, 'disaster': 75, 'but': 75, 'home': 74, 'suicide': 72, 'crash': 71, 'train': 70, 'MH370': 70, 'into': 69, 'no': 69, 'just': 67, 'will': 66, 'More': 66, 'out': 65, 'about': 65, 'bomb': 65, 'year': 64, '

In [85]:
def top_words(word_counts, n=20):
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    top_n = sorted_word_counts[:n]
    return top_n

disasterTweets_top = top_words(disasterTweets_count)
normalTweets_top = top_words(normalTweets_count)

print("Disaster Tweets TOP20:")
for word, count in disasterTweets_top:
    print(f"{word}: {count}")

print("\nNormal Tweets TOP20:")
for word, count in normalTweets_top:
    print(f"{word}: {count}")


Disaster Tweets TOP20:
:: 3676
http: 2513
#: 1703
.: 1086
the: 1060
in: 1046
@: 905
a: 892
of: 890
?: 762
to: 710
...: 592
and: 469
-: 398
on: 392
for: 371
!: 330
I: 329
is: 315
': 303

Normal Tweets TOP20:
:: 3030
?: 2364
http: 2203
.: 1872
@: 1854
#: 1700
the: 1532
a: 1264
I: 1245
to: 1105
and: 845
!: 843
of: 840
in: 731
you: 622
is: 525
's: 504
it: 460
for: 455
my: 443


In [92]:
def extract_ngrams(tweet_list, n):
    # Tokenize each tweet into individual words and add it
    tweets_combined = ' '.join(tweet_list)
    # Tokenize the combined text into individual words
    words = tweets_combined.split()
    n_grams = list(ngrams(words, n))
    return n_grams

disaster_bigrams = extract_ngrams(disasterTweets['lemmatized_text'], 2)
disaster_trigrams = extract_ngrams(disasterTweets['lemmatized_text'], 3)

normal_bigrams = extract_ngrams(normalTweets['lemmatized_text'], 2)
normal_trigrams = extract_ngrams(normalTweets['lemmatized_text'], 3)

print("Disaster Tweets Bigram:")
print(disaster_bigrams[:10])

print("\nDisaster Tweets Trigrams:")
print(disaster_trigrams[:10])

print("\nNormal Tweets Bigram:")
print(normal_bigrams[:10])

print("\nNormal Tweets Trigrams:")
print(normal_trigrams[:10])

Disaster Tweets Bigram:
[('Our', 'Deeds'), ('Deeds', 'are'), ('are', 'the'), ('the', 'Reason'), ('Reason', 'of'), ('of', 'this'), ('this', '#'), ('#', 'earthquake'), ('earthquake', 'May'), ('May', 'ALLAH')]

Disaster Tweets Trigrams:
[('Our', 'Deeds', 'are'), ('Deeds', 'are', 'the'), ('are', 'the', 'Reason'), ('the', 'Reason', 'of'), ('Reason', 'of', 'this'), ('of', 'this', '#'), ('this', '#', 'earthquake'), ('#', 'earthquake', 'May'), ('earthquake', 'May', 'ALLAH'), ('May', 'ALLAH', 'Forgive')]

Normal Tweets Bigram:
[('What', "'s"), ("'s", 'up'), ('up', 'man'), ('man', '?'), ('?', 'I'), ('I', 'love'), ('love', 'fruit'), ('fruit', 'Summer'), ('Summer', 'is'), ('is', 'lovely')]

Normal Tweets Trigrams:
[('What', "'s", 'up'), ("'s", 'up', 'man'), ('up', 'man', '?'), ('man', '?', 'I'), ('?', 'I', 'love'), ('I', 'love', 'fruit'), ('love', 'fruit', 'Summer'), ('fruit', 'Summer', 'is'), ('Summer', 'is', 'lovely'), ('is', 'lovely', 'My')]


In [95]:
# extract and count n-grams
def extract_and_count_ngrams(tweet_list, n):
    ngram_counts = Counter(chain.from_iterable(ngrams(tweet.split(), n) for tweet in tweet_list))
    return ngram_counts

disaster_bigram_counts = extract_and_count_ngrams(disasterTweets['lemmatized_text'], 2)
disaster_trigram_counts = extract_and_count_ngrams(disasterTweets['lemmatized_text'], 3)

normal_bigram_counts = extract_and_count_ngrams(normalTweets['lemmatized_text'], 2)
normal_trigram_counts = extract_and_count_ngrams(normalTweets['lemmatized_text'], 3)

# top n n-grams by occurrence
def top_n_ngrams(ngram_counts, n=20):
    return ngram_counts.most_common(n)

print("Top 20 Bigrams in Disaster Tweets:")
print(top_n_ngrams(disaster_bigram_counts))
print("\nTop 20 Trigrams in Disaster Tweets:")
print(top_n_ngrams(disaster_trigram_counts))

print("\nTop 20 Bigrams in Normal Tweets:")
print(top_n_ngrams(normal_bigram_counts))
print("\nTop 20 Trigrams in Normal Tweets:")
print(top_n_ngrams(normal_trigram_counts))

Top 20 Bigrams in Disaster Tweets:
[(('http', ':'), 2513), (('...', 'http'), 433), (('?', '?'), 402), (('amp', ';'), 135), (('&', 'amp'), 133), (('in', 'the'), 132), (('.', 'http'), 114), (('of', 'the'), 111), ((':', 'http'), 91), (('via', '@'), 88), (('.', '#'), 81), (('-', 'http'), 79), (('!', '!'), 79), (('?', '#'), 55), ((':', 'More'), 54), (('#', '?'), 50), (('in', '#'), 47), (('in', 'a'), 47), (('?', 'http'), 46), (('on', 'the'), 45)]

Top 20 Trigrams in Disaster Tweets:
[(('...', 'http', ':'), 433), (('?', '?', '?'), 250), (('&', 'amp', ';'), 133), (('.', 'http', ':'), 114), ((':', 'http', ':'), 91), (('-', 'http', ':'), 79), (('#', '?', '?'), 49), (('?', 'http', ':'), 46), (('!', '!', '!'), 39), (('?', '?', '#'), 37), ((')', 'http', ':'), 36), (('!', 'http', ':'), 34), (('?', '#', '?'), 30), (('The', 'Latest', ':'), 30), (('Latest', ':', 'More'), 28), (('PKK', 'suicide', 'bomber'), 28), (('suicide', 'bomber', 'who'), 28), (('bomber', 'who', 'detonated'), 28), (('who', 'detonate

## Part B

In [99]:
nltk.download('stopwords')
stemmer = SnowballStemmer("english")

def preprocess_data(data):

    #removal of url
    text = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ',data)

    #decontraction
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)

    #removal of html tags
    text = re.sub(r'<.*?>',' ',text)

    # Match all digits in the string and replace them by empty string
    text = re.sub(r'[0-9]', '', text)
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',text)

    # filtering out miscellaneous text.
    text = re.sub('[^a-zA-Z]',' ',text)
    text = re.sub(r"\([^()]*\)", "", text)

    # remove mentions
    text = re.sub('@\S+', '', text)

    # remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)


    # Lowering all the words in text
    text = text.lower()
    text = text.split()

    text = [stemmer.stem(words) for words in text if words not in stopwords.words('english')]

    # Removal of words with length<2
    text = [i for i in text if len(i)>2]
    text = ' '.join(text)
    return text

disasterTweets["Cleaned_text"] = disasterTweets["text"].apply(preprocess_data)
normalTweets["Cleaned_text"] = normalTweets["text"].apply(preprocess_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

<ipython-input-99-e18ea0a466f2>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disasterTweets["Cleaned_text"] = disasterTweets["text"].apply(preprocess_data)
<ipython-input-99-e18ea0a466f2>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalTweets["Cleaned_text"] = normalTweets["text"].apply(preprocess_data)


In [100]:
disasterTweets.head()

,text,target,lemmatized_text,Cleaned_text
0,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this # earthquake ...,deed reason earthquak may allah forgiv
1,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask . Canada,forest fire near rong sask canada
2,All residents asked to 'shelter in place' are ...,1,All resident asked to 'shelter in place ' are ...,resid ask ishelt place notifi offic evacu shel...
3,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive # wildfire evacuation or...",peopl receiv wildfir evacu order california
4,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby # Alaska a ...,got sent photo rubi alaska smoke wildfir pour ...


In [101]:
normalTweets.head()

,text,target,lemmatized_text,Cleaned_text
15,What's up man?,0,What 's up man ?,man
16,I love fruits,0,I love fruit,love fruit
17,Summer is lovely,0,Summer is lovely,summer love
18,My car is so fast,0,My car is so fast,car fast
19,What a goooooooaaaaaal!!!!!!,0,What a goooooooaaaaaal ! ! ! ! ! !,goooooooaaaaaal


# **Part C**

In [110]:
combined_df = pd.concat([disasterTweets, normalTweets], ignore_index=True)

combined_df.drop(columns=["text", "lemmatized_text"], inplace=True)

combined_df.rename(columns={"Cleaned_text": "text"}, inplace=True)
combined_df.head()

,target,text
0,1,deed reason earthquak may allah forgiv
1,1,forest fire near rong sask canada
2,1,resid ask ishelt place notifi offic evacu shel...
3,1,peopl receiv wildfir evacu order california
4,1,got sent photo rubi alaska smoke wildfir pour ...


In [106]:
test = pd.read_csv('/content/drive/MyDrive/NLP/lab4/test.csv')

In [108]:
test = test[['text']]
test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [111]:
X_train = combined_df['text']
y_train = combined_df['target']
X_test = test['text']

In [112]:
vectorizers = {
    'max_features_100': CountVectorizer(max_features=100),
    'max_features_1000': CountVectorizer(max_features=1000)
}

In [114]:
for name, vectorizer in vectorizers.items():
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_vec, y_train, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train_split, y_train_split)

    y_pred = model.predict(X_val_split)

    accuracy = accuracy_score(y_val_split, y_pred)
    f1 = f1_score(y_val_split, y_pred, average='weighted')
    recall = recall_score(y_val_split, y_pred, average='weighted')

    print(f"Performance for {name}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Recall: {recall}")

LogisticRegression()

Performance for max_features_100:
Accuracy: 0.7137229152987524
F1 Score: 0.7041464347586397
Recall: 0.7137229152987524


LogisticRegression()

Performance for max_features_1000:
Accuracy: 0.7912015758371634
F1 Score: 0.7888724100567592
Recall: 0.7912015758371634
